In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ir-a3-final-data/Electronics_5.json
/kaggle/input/ir-a3-final-data/meta_Electronics.json
/kaggle/input/ir-a3-data-csv/review_data.csv
/kaggle/input/ir-a3-data-csv/metadata_data.csv


In [2]:
import pandas as pd
import re
import unicodedata
import string
from nltk.corpus import wordnet
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordcloud import WordCloud

## **Converting the Product Review Data into a .csv file**

In [3]:
# #Splitting the Electronics.json and reading in chunks 
# with open(r"/kaggle/input/ir-a3-data/Electronics_5.json", "r") as fptr:
#     # Define chunk size
#     chunk_size = 20000  

#     review_data = []

#     for chunk in pd.read_json(fptr, lines=True, chunksize=chunk_size):
# #         print(chunk.head())
#         review_data.append(chunk)       


In [4]:
# review_data[0].columns

In [5]:
# #Combining all chunks of the Product review data
# review_df = pd.concat(review_data)

In [6]:
# #Saving the Dataframe as a CSV file
# review_df.to_csv("review_Data.csv")

## **Converting the Product MetaData into a .csv file**

In [7]:
# #Splitting the meta_Electronics.json and reading in chunks

# with open(r"/kaggle/input/ir-a3-final-data/meta_Electronics.json", "r") as fptr:
#     chunk_size = 20000 

#     metadata_df = []

#     for chunk in pd.read_json(fptr, lines=True, chunksize=chunk_size):
# #         print(chunk.head())
#         metadata_df.append(chunk)

In [8]:
# metadata_df[0].columns

In [9]:
# len(metadata_df)

In [10]:
# #Keeping only Data for Headphones
# for i in range(len(metadata_df)):
#     metadata_df[i] = metadata_df[i][metadata_df[i]['title'].str.contains('Headphones', case=False)]

In [11]:
# len(metadata_df)

In [12]:
# #Combine all the dataframes from metadata_df
# final_metadata = pd.concat(metadata_df)

In [13]:
# #Save this file as .csv file
# final_metadata.to_csv("metadata_data.csv", escapechar='\\')

## **Loading the Product Metadata & Review Data**

In [14]:
metadata_data = pd.read_csv(r"/kaggle/input/ir-a3-data-csv/metadata_data.csv")
review_data = pd.read_csv(r"/kaggle/input/ir-a3-data-csv/review_data.csv")

/tmp/ipykernel_34/2097714006.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  review_data = pd.read_csv(r"/kaggle/input/ir-a3-data-csv/review_data.csv")


In [15]:
print(metadata_data.shape)
print(review_data.shape)

(18210, 20)
(6739590, 13)


In [16]:
print(metadata_data.columns)

Index(['Unnamed: 0', 'category', 'tech1', 'description', 'fit', 'title',
       'also_buy', 'tech2', 'brand', 'feature', 'rank', 'also_view',
       'main_cat', 'similar_item', 'date', 'price', 'asin', 'imageURL',
       'imageURLHighRes', 'details'],
      dtype='object')


In [17]:
print(review_data.columns)

Index(['Unnamed: 0', 'overall', 'vote', 'verified', 'reviewTime', 'reviewerID',
       'asin', 'style', 'reviewerName', 'reviewText', 'summary',
       'unixReviewTime', 'image'],
      dtype='object')


In [18]:
metadata_data.head()

,Unnamed: 0,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,8,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,"['', '<b>True High Definition Sound:</b><br>Wi...",NaN,Wireless Bluetooth Headphones Earbuds with Mic...,[],NaN,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,['>#950 in Cell Phones & Accessories (See Top ...,[],Home Audio & Theater,NaN,"October 23, 2017",$7.99,0132492776,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
1,47,"['Electronics', 'Headphones']",NaN,['Use these high quality headphones for intern...,NaN,Polaroid Pbm2200 PC / Gaming Stereo Headphones...,[],NaN,Polaroid,"['Ideal for PC Internet chatting, PC / Console...","['>#3,548,269 in Cell Phones &amp; Accessories...",[],All Electronics,NaN,"December 13, 2012",NaN,0558835155,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
2,132,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,"['', '<b>True High Definition Sound:</b><br>Wi...",NaN,Bluetooth Workout Headphones for Running and G...,[],NaN,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#4,626,934 in Cell Phones & Accessories (Se...",[],Home Audio & Theater,NaN,"December 28, 2015",NaN,0692206280,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
3,223,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,"['', '<b>True High Definition Sound:</b><br />...",NaN,Bluetooth Workout Headphones for Running and G...,[],NaN,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#2,654,020 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,NaN,"October 18, 2015",NaN,0983629269,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
4,229,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,"['', '<b>True High Definition Sound:</b><br />...",NaN,Bluetooth Workout Headphones for Running and G...,[],NaN,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#5,289,289 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,NaN,"April 26, 2013",NaN,0985262788,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN


In [19]:
review_data.head()

,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,0,5,67.0,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,NaN
1,1,3,5.0,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,NaN
2,2,5,4.0,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,NaN
3,3,5,13.0,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,NaN
4,4,3,8.0,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,NaN


## Remove unecessary column

In [20]:
metadata_data = metadata_data.drop('Unnamed: 0', axis=1)
metadata_data.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [21]:
review_data = review_data.drop('Unnamed: 0', axis=1)
review_data.columns

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],
      dtype='object')

## Remove Null Values with zero

In [22]:
metadata_data.fillna(0, inplace=True)
metadata_data

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br>Wi...",0.0,Wireless Bluetooth Headphones Earbuds with Mic...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,['>#950 in Cell Phones & Accessories (See Top ...,[],Home Audio & Theater,0,"October 23, 2017",$7.99,0132492776,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
1,"['Electronics', 'Headphones']",0,['Use these high quality headphones for intern...,0.0,Polaroid Pbm2200 PC / Gaming Stereo Headphones...,[],0,Polaroid,"['Ideal for PC Internet chatting, PC / Console...","['>#3,548,269 in Cell Phones &amp; Accessories...",[],All Electronics,0,"December 13, 2012",0,0558835155,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
2,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br>Wi...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#4,626,934 in Cell Phones & Accessories (Se...",[],Home Audio & Theater,0,"December 28, 2015",0,0692206280,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
3,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br />...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#2,654,020 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,0,"October 18, 2015",0,0983629269,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
4,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br />...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#5,289,289 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,0,"April 26, 2013",0,0985262788,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18205,"['Electronics', 'Headphones', 'Earbud Headphon...",0,[],0.0,"Bluetooth Headphones,Yostyle Wireless Headphon...",['B01I0U77O8'],0,Yostyle,"['With Bluetooth V4.1, advanced apt-X technolo...","['>#186,311 in Cell Phones & Accessories (See ...","['B01I0U77O8', 'B010RQWSE2', 'B00PGJWYJ0', 'B0...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",$9.91,B01HJ8E11E,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{}
18206,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>Specification</b></br> Driver: 5mm</b...",0.0,MAXROCK Noise Isolating Sleeping Headphones Ea...,[],0,MAXROCK,"['Unique patented silicone design headphones, ...","['>#21,087 in Musical Instruments (See Top 100...","['B071WRSL38', 'B00XCDOGY8', 'B00V9FN1R4', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",0,B01HJAVYDU,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{}
18207,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"[""About the product Rhapsody & Mogan H9 is a ...",0.0,"Wireless Bluetooth Headset, HandsFree Wireless...",[],0,snorain,['COMFORTABLE CUSTOM FIT Rhapsody & Mogan noi...,"['>#343,752 in Cell Phones & Accessories (See ...","['B01D3QZB2Y', 'B079GFF4HZ', 'B00XBZY0EI', 'B0...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",0,B01HJAPNHI,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/i

In [23]:
review_data.fillna(0, inplace=True)
review_data

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,67.0,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,0
1,3,5.0,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,0
2,5,4.0,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,0
3,5,13.0,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,0
4,3,8.0,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6739585,4,0,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,0,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,0
6739586,4,0,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,0,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,0
6739587,5,2.0,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,0,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,0
6739588,5,2.0,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,0,Faith,Works well.,Five Stars,1480377600,0


## Remove Duplicate entries

In [24]:
metadata_data.drop_duplicates(inplace=True)
# metadata_data

In [25]:
review_data.drop_duplicates(inplace=True)
# review_data

In [26]:
review_data

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,67.0,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,0
1,3,5.0,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,0
2,5,4.0,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,0
3,5,13.0,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,0
4,3,8.0,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6739585,4,0,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,0,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,0
6739586,4,0,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,0,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,0
6739587,5,2.0,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,0,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,0
6739588,5,2.0,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,0,Faith,Works well.,Five Stars,1480377600,0


In [27]:
review_data.reset_index(inplace=True)

In [28]:
review_data.drop(columns=["index"], inplace=True)


In [30]:
# review_data.drop(columns=["level_0"], inplace=True)
review_data

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,67.0,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,0
1,3,5.0,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,0
2,5,4.0,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,0
3,5,13.0,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,0
4,3,8.0,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6557045,4,0,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,0,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,0
6557046,4,0,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,0,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,0
6557047,5,2.0,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,0,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,0
6557048,5,2.0,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,0,Faith,Works well.,Five Stars,1480377600,0


In [31]:
metadata_data

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br>Wi...",0.0,Wireless Bluetooth Headphones Earbuds with Mic...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,['>#950 in Cell Phones & Accessories (See Top ...,[],Home Audio & Theater,0,"October 23, 2017",$7.99,0132492776,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
1,"['Electronics', 'Headphones']",0,['Use these high quality headphones for intern...,0.0,Polaroid Pbm2200 PC / Gaming Stereo Headphones...,[],0,Polaroid,"['Ideal for PC Internet chatting, PC / Console...","['>#3,548,269 in Cell Phones &amp; Accessories...",[],All Electronics,0,"December 13, 2012",0,0558835155,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
2,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br>Wi...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#4,626,934 in Cell Phones & Accessories (Se...",[],Home Audio & Theater,0,"December 28, 2015",0,0692206280,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
3,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br />...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#2,654,020 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,0,"October 18, 2015",0,0983629269,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
4,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br />...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#5,289,289 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,0,"April 26, 2013",0,0985262788,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18205,"['Electronics', 'Headphones', 'Earbud Headphon...",0,[],0.0,"Bluetooth Headphones,Yostyle Wireless Headphon...",['B01I0U77O8'],0,Yostyle,"['With Bluetooth V4.1, advanced apt-X technolo...","['>#186,311 in Cell Phones & Accessories (See ...","['B01I0U77O8', 'B010RQWSE2', 'B00PGJWYJ0', 'B0...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",$9.91,B01HJ8E11E,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{}
18206,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>Specification</b></br> Driver: 5mm</b...",0.0,MAXROCK Noise Isolating Sleeping Headphones Ea...,[],0,MAXROCK,"['Unique patented silicone design headphones, ...","['>#21,087 in Musical Instruments (See Top 100...","['B071WRSL38', 'B00XCDOGY8', 'B00V9FN1R4', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",0,B01HJAVYDU,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{}
18207,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"[""About the product Rhapsody & Mogan H9 is a ...",0.0,"Wireless Bluetooth Headset, HandsFree Wireless...",[],0,snorain,['COMFORTABLE CUSTOM FIT Rhapsody & Mogan noi...,"['>#343,752 in Cell Phones & Accessories (See ...","['B01D3QZB2Y', 'B079GFF4HZ', 'B00XBZY0EI', 'B0...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",0,B01HJAPNHI,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/i

In [32]:
metadata_data.reset_index(inplace=True)
metadata_data.drop(columns="index", inplace=True)
metadata_data

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br>Wi...",0.0,Wireless Bluetooth Headphones Earbuds with Mic...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,['>#950 in Cell Phones & Accessories (See Top ...,[],Home Audio & Theater,0,"October 23, 2017",$7.99,0132492776,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
1,"['Electronics', 'Headphones']",0,['Use these high quality headphones for intern...,0.0,Polaroid Pbm2200 PC / Gaming Stereo Headphones...,[],0,Polaroid,"['Ideal for PC Internet chatting, PC / Console...","['>#3,548,269 in Cell Phones &amp; Accessories...",[],All Electronics,0,"December 13, 2012",0,0558835155,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
2,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br>Wi...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#4,626,934 in Cell Phones & Accessories (Se...",[],Home Audio & Theater,0,"December 28, 2015",0,0692206280,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
3,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br />...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#2,654,020 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,0,"October 18, 2015",0,0983629269,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
4,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>True High Definition Sound:</b><br />...",0.0,Bluetooth Workout Headphones for Running and G...,[],0,Enter The Arena,['Superb Sound Quality: Plays crystal clear au...,"['>#5,289,289 in Cell Phones &amp; Accessories...",[],Home Audio &amp; Theater,0,"April 26, 2013",0,0985262788,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17805,"['Electronics', 'Headphones', 'Earbud Headphon...",0,[],0.0,"Bluetooth Headphones,Yostyle Wireless Headphon...",['B01I0U77O8'],0,Yostyle,"['With Bluetooth V4.1, advanced apt-X technolo...","['>#186,311 in Cell Phones & Accessories (See ...","['B01I0U77O8', 'B010RQWSE2', 'B00PGJWYJ0', 'B0...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",$9.91,B01HJ8E11E,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{}
17806,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"['', '<b>Specification</b></br> Driver: 5mm</b...",0.0,MAXROCK Noise Isolating Sleeping Headphones Ea...,[],0,MAXROCK,"['Unique patented silicone design headphones, ...","['>#21,087 in Musical Instruments (See Top 100...","['B071WRSL38', 'B00XCDOGY8', 'B00V9FN1R4', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",0,B01HJAVYDU,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,{}
17807,"['Electronics', 'Headphones', 'Earbud Headphon...",0,"[""About the product Rhapsody & Mogan H9 is a ...",0.0,"Wireless Bluetooth Headset, HandsFree Wireless...",[],0,snorain,['COMFORTABLE CUSTOM FIT Rhapsody & Mogan noi...,"['>#343,752 in Cell Phones & Accessories (See ...","['B01D3QZB2Y', 'B079GFF4HZ', 'B00XBZY0EI', 'B0...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 25, 2016",0,B01HJAPNHI,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/i

In [33]:
print(metadata_data.columns, metadata_data.shape)
print(review_data.columns, review_data.shape)

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object') (17810, 19)
Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],
      dtype='object') (6557050, 12)


## **Keep those rows in the Product review_data only where the ASIN is present in the Product Metadata_data**

In [34]:
selected_asin_vals = metadata_data["asin"]

In [35]:
review_data = review_data[review_data["asin"].isin(selected_asin_vals)]
review_data

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
4823,5,2.0,True,"02 28, 2013",A15U5NUS1EY7IQ,B000001OMI,{'Format:': ' Personal Computers'},S. Bommarito,I'm getting listening stations going in severa...,Works well,1362009600,0
4824,5,5.0,True,"10 17, 2012",AHGGNAIT6BH5X,B000001OMI,0,Edward H,Needed Headset for watching and editing videos...,HP 200 Stereo Headset,1350432000,0
4825,4,0,False,"09 27, 2012",A33HUURDJ4DVG3,B000001OMI,{'Format:': ' Personal Computers'},HB,I was out of town for a month and needed a pai...,"For the price, you cannot beat them",1348704000,0
4826,5,4.0,True,"11 27, 2008",A680RUE1FDO8B,B000001OMI,{'Format:': ' Personal Computers'},Jerry Saperstein,Obviously these headphones are not for the aud...,Excellent day-to-day headphones,1227744000,0
4827,3,2.0,False,"12 24, 2007",A3OLAZKM77XR6P,B000001OMI,{'Format:': ' Personal Computers'},Tennis Ace,I bought these headphones at a Tower Records a...,Foldable but didn't last long,1198454400,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6556912,5,0,True,"05 4, 2018",A3E7OV7IR941JI,B01HI7D4VY,0,JC,I used these for tail light replacement bulbs ...,Excellent LEDS,1525392000,0
6556913,3,0,True,"03 14, 2018",A26I2RPRWOGD2V,B01HI7D4VY,0,Ray,Not the brightest bulb on the block.,Not the brightest bulb on the block.,1520985600,0
6557019,4,0,True,"09 22, 2018",A3VA3VK4PO1JD,B01HJ8E11E,{'Color:': ' White'},CD,I have only used these for 1 week at the time ...,They work...,1537574400,0
6557020,3,0,True,"09 12, 2018",A11TVS6FKXS80H,B01HJ8E11E,{'Color:': ' White'},Jay Salamon,"The product works great, but when it gets down...",30% charge shuts the device off,1536710400,0


In [36]:
review_data.reset_index(inplace=True)



In [37]:
# review_data.drop(columns=["index"], inplace=True)
# review_data.drop(columns=["level_0"], inplace=True)
review_data

,index,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,4823,5,2.0,True,"02 28, 2013",A15U5NUS1EY7IQ,B000001OMI,{'Format:': ' Personal Computers'},S. Bommarito,I'm getting listening stations going in severa...,Works well,1362009600,0
1,4824,5,5.0,True,"10 17, 2012",AHGGNAIT6BH5X,B000001OMI,0,Edward H,Needed Headset for watching and editing videos...,HP 200 Stereo Headset,1350432000,0
2,4825,4,0,False,"09 27, 2012",A33HUURDJ4DVG3,B000001OMI,{'Format:': ' Personal Computers'},HB,I was out of town for a month and needed a pai...,"For the price, you cannot beat them",1348704000,0
3,4826,5,4.0,True,"11 27, 2008",A680RUE1FDO8B,B000001OMI,{'Format:': ' Personal Computers'},Jerry Saperstein,Obviously these headphones are not for the aud...,Excellent day-to-day headphones,1227744000,0
4,4827,3,2.0,False,"12 24, 2007",A3OLAZKM77XR6P,B000001OMI,{'Format:': ' Personal Computers'},Tennis Ace,I bought these headphones at a Tower Records a...,Foldable but didn't last long,1198454400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260977,6556912,5,0,True,"05 4, 2018",A3E7OV7IR941JI,B01HI7D4VY,0,JC,I used these for tail light replacement bulbs ...,Excellent LEDS,1525392000,0
260978,6556913,3,0,True,"03 14, 2018",A26I2RPRWOGD2V,B01HI7D4VY,0,Ray,Not the brightest bulb on the block.,Not the brightest bulb on the block.,1520985600,0
260979,6557019,4,0,True,"09 22, 2018",A3VA3VK4PO1JD,B01HJ8E11E,{'Color:': ' White'},CD,I have only used these for 1 week at the time ...,They work...,1537574400,0
260980,6557020,3,0,True,"09 12, 2018",A11TVS6FKXS80H,B01HJ8E11E,{'Color:': ' White'},Jay Salamon,"The product works great, but when it gets down...",30% charge shuts the device off,1536710400,0


In [38]:
print(metadata_data.shape)
print(review_data.shape)

(17810, 19)
(260982, 13)


# **4. Descriptive Statistics of the Product**

## a) Number of reviews for each ASIN

In [39]:
reviews_asin_wise = {}

for idx, row in review_data.iterrows():
#     print(idx, row)
#     print()
    if row['asin'] not in reviews_asin_wise:
        reviews_asin_wise[row['asin']] = 1
    else:
        reviews_asin_wise[row['asin']] += 1

        


In [40]:
print(reviews_asin_wise)

{'B000001OMI': 41, 'B00000DMA3': 11, 'B00000J1FV': 6, 'B00000J1GA': 12, 'B00000JBHP': 66, 'B00001OPK7': 7, 'B00001P505': 198, 'B00001P4ZH': 1158, 'B00001P4XA': 380, 'B00001P4ZR': 9, 'B00001W0D5': 8, 'B00001W0DG': 14, 'B00001W0DC': 26, 'B00001WRSJ': 1104, 'B00001W0DD': 34, 'B00001W0D4': 10, 'B00004LC7Z': 8, 'B00004SD87': 8, 'B00004SD88': 12, 'B00004T8R2': 1707, 'B00004T8R4': 6, 'B00004TZJQ': 7, 'B00004TZJI': 6, 'B00004TZJJ': 5, 'B00004Z0BP': 6, 'B00004Z5D1': 415, 'B00004Z7G9': 118, 'B00004Z6Q6': 28, 'B00005105L': 56, 'B000056SJT': 10, 'B000056SJU': 6, 'B00005BYER': 7, 'B00005K3LZ': 5, 'B00005N6KF': 16, 'B00005N6KG': 456, 'B00005N9D2': 72, 'B00005N9D3': 542, 'B00005OTG6': 35, 'B00005QBU9': 374, 'B00005QBUU': 102, 'B00005R8Q1': 7, 'B00005R8PV': 6, 'B00005R8PW': 8, 'B00005RFD3': 370, 'B00005T3SQ': 8, 'B000065BP9': 567, 'B000065BPA': 42, 'B000065BPC': 30, 'B000065BPB': 828, 'B000066CCU': 46, 'B000067FCS': 15, 'B000067G0J': 89, 'B000067SBB': 30, 'B000068IFM': 19, 'B00006B9W9': 13, 'B00006HOG

## b) Average Rating score for each ASIN

In [41]:
avg_rating_asin_wise = {}

for idx, row in review_data.iterrows():
    if row["asin"] not in avg_rating_asin_wise:
        avg_rating_asin_wise[row["asin"]] = []
        avg_rating_asin_wise[row["asin"]].append(int(row["overall"]))
        
    else:
        avg_rating_asin_wise[row["asin"]].append(int(row["overall"]))

        
for asin in avg_rating_asin_wise:
    avg_rating_asin_wise[asin]=sum(avg_rating_asin_wise[asin])/len(avg_rating_asin_wise[asin])

In [42]:
print(avg_rating_asin_wise)

{'B000001OMI': 3.975609756097561, 'B00000DMA3': 4.0, 'B00000J1FV': 3.0, 'B00000J1GA': 3.25, 'B00000JBHP': 4.2272727272727275, 'B00001OPK7': 3.857142857142857, 'B00001P505': 4.318181818181818, 'B00001P4ZH': 4.442141623488774, 'B00001P4XA': 3.9526315789473685, 'B00001P4ZR': 4.0, 'B00001W0D5': 3.75, 'B00001W0DG': 3.7857142857142856, 'B00001W0DC': 4.115384615384615, 'B00001WRSJ': 4.618659420289855, 'B00001W0DD': 3.676470588235294, 'B00001W0D4': 4.4, 'B00004LC7Z': 3.25, 'B00004SD87': 3.25, 'B00004SD88': 4.25, 'B00004T8R2': 4.330404217926186, 'B00004T8R4': 3.8333333333333335, 'B00004TZJQ': 3.5714285714285716, 'B00004TZJI': 4.166666666666667, 'B00004TZJJ': 3.4, 'B00004Z0BP': 4.333333333333333, 'B00004Z5D1': 4.506024096385542, 'B00004Z7G9': 4.245762711864407, 'B00004Z6Q6': 3.9285714285714284, 'B00005105L': 3.8392857142857144, 'B000056SJT': 3.8, 'B000056SJU': 2.8333333333333335, 'B00005BYER': 3.4285714285714284, 'B00005K3LZ': 2.8, 'B00005N6KF': 3.6875, 'B00005N6KG': 4.109649122807017, 'B00005N9

## Number of unique products

In [43]:
#no. of unique products

num_of_unique_prods = review_data["asin"].nunique()
print(num_of_unique_prods)

5393


## Number of Good Ratings and Bad Ratings 

In [44]:
good_ratings = []
bad_ratings = []


for idx, row in review_data.iterrows():
    if row["overall"]>=3:
        good_ratings.append((row["overall"], row["reviewText"]))
        
    else:
        bad_ratings.append((row["overall"], row["reviewText"]))

In [45]:
print("Number of Good Ratings: {}".format(len(good_ratings)))
print("Number of Bad Ratings: {}".format(len(bad_ratings)))

Number of Good Ratings: 222538
Number of Bad Ratings: 38444


In [46]:
print("Number of Reviews for Good Rating: {}".format(len(good_ratings)))
print("Number of Reviews for Bad Rating: {}".format(len(bad_ratings)))

Number of Reviews for Good Rating: 222538
Number of Reviews for Bad Rating: 38444


# **5. Preprocess the Text**

In [47]:
!pip install nltk
!pip install textblob
!pip install wordcloud
!pip install --upgrade wordcloud

INFO: pip is looking at multiple versions of textblob to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement nltk>=3.8 (from textblob) (from versions: none)
ERROR: No matching distribution found for nltk>=3.8


In [48]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

a) Remove HTML Tags

In [49]:
def remove_html_tags(txt):
    if isinstance(txt, str):
        soup = BeautifulSoup(txt, "html.parser")
        return soup.get_text()
    
    return ""
    

b) Remove Accented Characters

In [50]:
def remove_accented_chars(txt):
    txt = unicodedata.normalize('NFKD', txt).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    return txt

    

d) Remove Special Characters

In [51]:
def remove_special_chars(txt):
    txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    
    return txt

e) Lemmatization 

In [52]:
lemmatizer = WordNetLemmatizer()

def apply_lemmatization(txt):
    tokens = txt.split()
    
    processed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_txt = " ".join(processed_tokens)
    
    return lemmatized_txt

f) Normalize Text

In [53]:
stop_words = set(stopwords.words('english'))


def text_normalizer(txt):
    
    processed_txt = txt.lower()
    processed_txt = processed_txt.translate(str.maketrans('', '', string.punctuation))
    processed_txt = re.sub(r'\d+', '', processed_txt)
    
    tokens = word_tokenize(processed_txt)
    processed_tokens = [word for word in tokens if word not in stop_words]
    normalized_txt = ' '.join(processed_tokens)

    return normalized_txt

## Preprocessing the Text

In [54]:
def preprocess_txt(txt):
    
    txt = remove_html_tags(txt)
    txt = remove_accented_chars(txt)
    txt = remove_special_chars(txt)
#     txt = apply_lemmatization(txt)
    
    txt = text_normalizer(txt)
    
    return txt

In [55]:

review_data['reviewText'] = review_data['reviewText'].apply(preprocess_txt)

/tmp/ipykernel_34/1081606578.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(txt, "html.parser")
/tmp/ipykernel_34/1081606578.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(txt, "html.parser")
/tmp/ipykernel_34/2441817639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_data['reviewText'] = review_data['reviewText'].apply(preprocess_txt)


In [78]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split

In [56]:
#Initialize the Hashing vectorizer
hash_vectorizer = HashingVectorizer(n_features=1000) 

# Transfroming the review text using Hash vectorizer
hashed_features = hash_vectorizer.fit_transform(review_data['reviewText'])

## 8. Classifying Ratings

In [82]:
def categorize_rating(rating):
    if rating > 3:
        return 'Good'
    elif rating == 3:
        return 'Average'
    else:
        return 'Bad'
    

In [83]:
review_data['ratingClass'] = review_data['overall'].apply(categorize_rating)

/tmp/ipykernel_34/2263612672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_data['ratingClass'] = review_data['overall'].apply(categorize_rating)


In [59]:
review_data.head()

,index,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,ratingClass
0,4823,5,2.0,True,"02 28, 2013",A15U5NUS1EY7IQ,B000001OMI,{'Format:': ' Personal Computers'},S. Bommarito,im getting listening stations going several el...,Works well,1362009600,0,Good
1,4824,5,5.0,True,"10 17, 2012",AHGGNAIT6BH5X,B000001OMI,0,Edward H,needed headset watching editing videos late ni...,HP 200 Stereo Headset,1350432000,0,Good
2,4825,4,0,False,"09 27, 2012",A33HUURDJ4DVG3,B000001OMI,{'Format:': ' Personal Computers'},HB,town month needed pair headphones walking want...,"For the price, you cannot beat them",1348704000,0,Good
3,4826,5,4.0,True,"11 27, 2008",A680RUE1FDO8B,B000001OMI,{'Format:': ' Personal Computers'},Jerry Saperstein,obviously headphones audiophile person expects...,Excellent day-to-day headphones,1227744000,0,Good
4,4827,3,2.0,False,"12 24, 2007",A3OLAZKM77XR6P,B000001OMI,{'Format:': ' Personal Computers'},Tennis Ace,bought headphones tower records years ago nice...,Foldable but didn't last long,1198454400,0,Average


## Hash Vectorizer

In [60]:
X = hashed_features
y = review_data['ratingClass']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [61]:
print("Training set size (X, y): {}, {}".format(X_train.shape, y_train.shape))
print("Testing set size (X, y): {}, {}".format(X_test.shape, y_test.shape))

Training set size (X, y): (195736, 1000), (195736,)
Testing set size (X, y): (65246, 1000), (65246,)


# ML Models

In [79]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

## a) Model: Logistic Regression

In [76]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
    
#make predictions
y_pred = logreg.predict(X_test)

#classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Average       0.41      0.09      0.14      6857
         Bad       0.65      0.53      0.59      9652
        Good       0.84      0.96      0.89     48737

    accuracy                           0.80     65246
   macro avg       0.63      0.53      0.54     65246
weighted avg       0.76      0.80      0.77     65246



## b) Model: AdaBoost Classifier

In [75]:
adaboost = AdaBoostClassifier(n_estimators=10, random_state=42)
adaboost.fit(X_train, y_train)

#make predictions
y_pred = adaboost.predict(X_test)

#classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Average       0.29      0.00      0.00      6857
         Bad       0.55      0.20      0.29      9652
        Good       0.77      0.98      0.86     48737

    accuracy                           0.76     65246
   macro avg       0.54      0.39      0.39     65246
weighted avg       0.69      0.76      0.69     65246



## c) Model: Gaussian Naive Bayes

In [77]:
#Train the Model
Gauss_nb = GaussianNB()
Gauss_nb.fit(X_train.toarray(), y_train)

#make predictions
y_pred = Gauss_nb.predict(X_test.toarray())

#classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Average       0.18      0.47      0.26      6857
         Bad       0.39      0.57      0.46      9652
        Good       0.91      0.62      0.74     48737

    accuracy                           0.60     65246
   macro avg       0.49      0.56      0.49     65246
weighted avg       0.75      0.60      0.65     65246



## d) Model: Random Forest Classifier

In [69]:
#Train the Model
rand_forest = RandomForestClassifier(n_estimators=50, max_depth=5)
rand_forest.fit(X_train, y_train)

#make predictions
y_pred = rand_forest.predict(X_test)

#classification report
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     Average       0.00      0.00      0.00      6857
         Bad       0.00      0.00      0.00      9652
        Good       0.75      1.00      0.86     48737

    accuracy                           0.75     65246
   macro avg       0.25      0.33      0.29     65246
weighted avg       0.56      0.75      0.64     65246



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## e) Model: MLP (Mulit-Layer Perceptron)

In [71]:
#Train the Model
mlp = MLPClassifier(learning_rate_init=0.01, max_iter=10, activation='relu', hidden_layer_sizes=(256,32), verbose=True)
mlp.fit(X_train, y_train)


#make predictions
y_pred = mlp.predict(X_test)

#classification report
print(classification_report(y_test, y_pred))

Iteration 1, loss = 0.50799186
Iteration 2, loss = 0.45622547
Iteration 3, loss = 0.40710980
Iteration 4, loss = 0.35130905
Iteration 5, loss = 0.29491877
Iteration 6, loss = 0.25135668
Iteration 7, loss = 0.21757479
Iteration 8, loss = 0.19415819
Iteration 9, loss = 0.17775332
Iteration 10, loss = 0.16636055


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

     Average       0.30      0.27      0.29      6857
         Bad       0.59      0.57      0.58      9652
        Good       0.88      0.89      0.88     48737

    accuracy                           0.78     65246
   macro avg       0.59      0.58      0.58     65246
weighted avg       0.77      0.78      0.78     65246

